# Shor's Algorithm

Factoring numbers is classically a very hard problem. This is the reason why the most used crittography systems (RSA and ECC) are based on the complexity of factoring numbers, as e.g 300 digits long numbers take hundreds of thousend of years to factor even for the most powerful classical computer.

<br>

Shor's algorithm is the quantum algorithm that can factor any number $N$ in less than polynomial time, approximately order $O(log^3N)$. The base idea of the algorithm relies on modular arithmetic, of wich we give an example. $\\$Suppose we want to factor the number 21, we have that the key to the solution is to find a number $x$ such that: 
$$x^2 \equiv 1 \space (mod \space 21)$$
In fact if we have for instance $x = 8$, it follows that 
$$8^2 \equiv 1 \space (mod \space 21)$$
$$8^2 - 1^2 \equiv 0 \space (mod \space 21)$$
so that:
$$(8 - 1)(8 + 1) \equiv 0 \space (mod \space 21)$$
Now, $7$ divides $21$, but $9$ doesn't, and this is why we need to take the great common divisor between $21$ and the resuls we get to compute the factorization:
$$gcd(21, 7) = 7 \space \space \space gcd(21, 9) = 3$$
and we have our factorization.
Point is that we need to find a non trivial square root of $1 \space (mod \space N)$ (obviously $x=\pm1$ are trivial). We can do this by picking a random number, let's say $x=2$, for wich we have:
$$2^0 \equiv 1 \space (mod \space 21)$$
$$2^1 \equiv 2 \space (mod \space 21)$$
$$2^2 \equiv 4 \space (mod \space 21)$$
$$2^3 \equiv 8 \space (mod \space 21)$$
$$\space\space2^4 \equiv 16 \space (mod \space 21)$$
$$\space\space2^5 \equiv 11 \space (mod \space 21)$$
$$2^6 \equiv 1 \space (mod \space 21)$$
but $2^6 = 8^2$, which is the solution we found above. In general we are looking for an even power of a random number $x$ such that:
$$(x^{r/2})^2 \equiv 1 \space (mod \space 21)$$
and:
$$x^{r/2} \neq \pm1 \space (mod \space 21)$$
In other words, the goal is to find the period $r$ of the table above, for if we can do that we can construct a non trivial square root of 1.

### Implementation

To implement the proces above we need to find a way to implement a gate $U$ such that:

$$ U |x> \equiv |ax \space (mod \space N)> $$

so that every state gets multiplied by $a \space (mod \space N)$, and for $a = 2 N = 21$:

$$ U|1> = |2 \space (mod \space 21)> $$
$$ U^{2}|1> = |4 \space (mod \space 21)> $$
$$ \vdots $$
$$ U^r|1> = |1 \space (mod \space 21)> $$

Now, the linear combination of these states is an eigenstate of $U$. We can then construct a useful eigenstate with different phases:

$$ |u_s> = \frac{1}{\sqrt{r}} \sum_{j=0}^{r-1} e^{-\frac{2\pi i j s}{r}}|a^j \space (mod \space N)> $$

so that:

$$ U|u_s> = e^{\frac{2\pi i s}{r}}|u_s> $$

for some $s \in \{0, r-1\}$, producing a different eigenvalue for each eigenstate. We can then apply quantum phase estimation to get the phase $\frac{s}{r}$. A more detailed explenation of this process can be found at https://arxiv.org/pdf/quant-ph/0205095.pdf, the circuit will look like:

![Shor's Algorithm](https://www.researchgate.net/profile/Juha-Vartiainen-2/publication/235574493/figure/fig1/AS:669038392926232@1536522591933/Quantum-circuit-for-Shors-algorithm.png)

### Qiskit

Qiskit offers an embedded implementation for the circuit, we can run the algorithm as follow:

In [9]:
from qiskit import IBMQ
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import Shor
from qiskit.providers.ibmq import least_busy

In [10]:
IBMQ.load_account()

ibmqfactory.load_account:WARNING:2021-09-04 11:38:10,899: Credentials are already in use. The existing account in the session will be replaced.


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [11]:
provider = IBMQ.get_provider(hub='ibm-q')
backend = provider.get_backend('ibmq_qasm_simulator')

In [12]:
factors = Shor(21)

result_dict = factors.run(QuantumInstance(backend, shots=1, skip_qobj_validation=False))
result = result_dict['factors']

print(result)

[[3, 7]]
